## Problem Statement : 
### Build a model that predicts total delivery time of order .

## Step 1: Data Gathering

In [2]:
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
path = r"C:\Users\SAMRUDHI\OneDrive\Desktop\DATA SCIENCE\Datasets\Dataset-Regression.csv"
df = pd.read_csv(path,na_values=['','NA','NaN','nan'],keep_default_na=False)
df.head()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration
0,1.0,22:24:17,23:27:16,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0
1,2.0,21:49:25,22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0
2,3.0,20:39:28,21:09:09,5477,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0
3,3.0,21:21:45,22:13:00,5477,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0
4,3.0,02:40:36,03:20:26,5477,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0


# Step 2: Perform data quality checks

In [4]:
df.shape

(197428, 16)

In [5]:
df.columns

Index(['market_id', 'created_at', 'actual_delivery_time', 'store_id',
       'store_primary_category', 'order_protocol', 'total_items', 'subtotal',
       'num_distinct_items', 'min_item_price', 'max_item_price',
       'total_onshift_dashers', 'total_busy_dashers',
       'total_outstanding_orders', 'estimated_order_place_duration',
       'estimated_store_to_consumer_driving_duration'],
      dtype='str')

In [6]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 197428 entries, 0 to 197427
Data columns (total 16 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   market_id                                     196441 non-null  float64
 1   created_at                                    197428 non-null  str    
 2   actual_delivery_time                          197421 non-null  str    
 3   store_id                                      197428 non-null  int64  
 4   store_primary_category                        192668 non-null  str    
 5   order_protocol                                196433 non-null  float64
 6   total_items                                   197428 non-null  int64  
 7   subtotal                                      197428 non-null  int64  
 8   num_distinct_items                            197428 non-null  int64  
 9   min_item_price                                197428 non-nu

In [7]:
# check for duplicates
df.duplicated().sum()

np.int64(0)

In [8]:
# drop duplicates
df = df.drop_duplicates()

In [9]:
df['created_at'] = pd.to_datetime(df['created_at'])
df['actual_delivery_time'] = pd.to_datetime(df['actual_delivery_time'])
df['delivery_duration'] = (df['actual_delivery_time'] - df['created_at']).dt.total_seconds()

In [10]:
df = df[df['delivery_duration'] > 0]

In [11]:
df['order_hour'] = df['created_at'].dt.hour
df['order_day'] = df['created_at'].dt.dayofweek
df['is_weekend'] = (df['order_day'] >= 5).astype(int)


In [12]:
df['dasher_utilization'] = (
    df['total_busy_dashers'] / df['total_onshift_dashers']
)


In [13]:
df.drop(['created_at', 'actual_delivery_time'], axis=1, inplace=True)

In [14]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)

,market_id,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,delivery_duration,order_hour,order_day,is_weekend,dasher_utilization
0,1.0,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0,3779.0,22,0,0,0.424242
1,2.0,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0,4024.0,21,0,0,2.000000
2,3.0,5477,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0,1781.0,20,0,0,0.000000
3,3.0,5477,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0,3075.0,21,0,0,1.000000
4,3.0,5477,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0,2390.0,2,0,0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197423,1.0,2956,fast,4.0,3,1389,3,345,649,17.0,17.0,23.0,251,331.0,3907.0,0,0,0,1.000000
197424,1.0,2956,fast,4.0,6,3010,4,405,825,12.0,11.0,14.0,251,915.0,3383.0,0,0,0,0.916667
197425,1.0,2956,fast,4.0,5,1836,3,300,399,39.0,41.0,40.0,251,795.0,3008.0,4,0,0,1.051282
197426,1.0,3630,sandwich,1.0,1,1175,1,535,535,7.0,7.0,12.0,446,384.0,3907.0,18,0,0,1.000000


## Step 3: Separate X and Y features

In [15]:
X = df.drop(columns='delivery_duration')
Y = df[['delivery_duration']]

In [16]:
X.head()

,market_id,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,order_hour,order_day,is_weekend,dasher_utilization
0,1.0,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0,22,0,0,0.424242
1,2.0,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0,21,0,0,2.000000
2,3.0,5477,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0,20,0,0,0.000000
3,3.0,5477,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0,21,0,0,1.000000
4,3.0,5477,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0,2,0,0,1.000000


In [17]:
Y.head()

,delivery_duration
0,3779.0
1,4024.0
2,1781.0
3,3075.0
4,2390.0


## Step 4: Data Cleaning and Data Preprocessing(Feature Scaling)

In [18]:
cat = list(X.columns[X.dtypes=='str'])
con = list(X.columns[X.dtypes!='str'])
print(cat,con,sep='\n')

['store_primary_category']
['market_id', 'store_id', 'order_protocol', 'total_items', 'subtotal', 'num_distinct_items', 'min_item_price', 'max_item_price', 'total_onshift_dashers', 'total_busy_dashers', 'total_outstanding_orders', 'estimated_order_place_duration', 'estimated_store_to_consumer_driving_duration', 'order_hour', 'order_day', 'is_weekend', 'dasher_utilization']


In [19]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

# Build the numerical pipeline
num_pipe = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler()
)

# Build the categorical pipeline
cat_pipe = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OneHotEncoder(sparse_output=False,handle_unknown='ignore')
)

# combine both the pipelines
pre = ColumnTransformer([
    ('cat',cat_pipe,cat),
    ('con',num_pipe,con)
]).set_output(transform='pandas')

# Apply fit and transform on X
X_pre = pre.fit_transform(X)
X_pre.head()

,cat__store_primary_category_afghan,cat__store_primary_category_african,cat__store_primary_category_alcohol,cat__store_primary_category_alcohol-plus-food,cat__store_primary_category_american,cat__store_primary_category_argentine,cat__store_primary_category_asian,cat__store_primary_category_barbecue,cat__store_primary_category_belgian,cat__store_primary_category_brazilian,...,con__max_item_price,con__total_onshift_dashers,con__total_busy_dashers,con__total_outstanding_orders,con__estimated_order_place_duration,con__estimated_store_to_consumer_driving_duration,con__order_hour,con__order_day,con__is_weekend,con__dasher_utilization
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.138530,-0.379308,-0.921010,-0.753909,1.523741,1.446638,1.664508,0.0,0.0,-1.365616
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.427363,-1.342792,-1.309400,-1.128432,1.523741,0.665392,1.545474,0.0,0.0,2.729715
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.324360,-1.342792,-1.374132,-1.167856,1.523741,0.665392,1.426440,0.0,0.0,-2.468205
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.144961,-1.342792,-1.341766,-1.128432,1.523741,-1.166653,1.545474,0.0,0.0,0.130755
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.786162,-1.192248,-1.179937,-0.990450,1.523741,0.482644,-0.716172,0.0,0.0,0.130755


# Step 5: Split the data into training and testing

In [20]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(X_pre,Y,train_size=0.8, random_state=21)

In [21]:
print(f'xtrain:{xtrain.shape}')
print(f'xtest:{xtest.shape}')
print(f'ytrain:{ytrain.shape}')
print(f'ytest:{ytest.shape}')

xtrain:(153167, 91)
xtest:(38292, 91)
ytrain:(153167, 1)
ytest:(38292, 1)


## Step 6: Build the model

In [22]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(xtrain,ytrain)

,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False


In [23]:
model.score(xtrain,ytrain)

0.2216331630515218

In [24]:
model.score(xtest,ytest)

0.22233717180128343

In [25]:
model.intercept_

array([2852.76042175])

In [26]:
model.coef_

array([[ 9.48111742e+01, -2.24768173e+02, -9.70841195e+01,
        -4.05816001e+02, -1.65881645e+01,  6.81614384e+01,
        -3.87892238e+00, -1.46142374e+02,  5.26273857e+02,
         3.34090802e+02, -5.38065699e+01, -2.39343470e+02,
         2.44355443e+02,  4.26111872e+01,  3.11715192e+01,
         7.90602365e+01,  8.07753130e+01,  1.96224703e+02,
        -5.96922902e+01,  1.10052325e+02, -2.78555919e+01,
        -4.39890537e+02,  2.64270423e+02,  2.56757357e+02,
         2.58380324e+01, -8.19401922e+01,  2.50475973e+01,
         1.81856224e+02,  2.16425714e+01,  5.15461039e+01,
        -1.70073190e+02, -1.16376991e+02, -1.61737125e+02,
        -3.74361203e+02,  1.01913449e+01, -8.61832258e+01,
         2.09671128e+01, -2.27373675e-12, -1.99957819e+02,
         2.45332727e-01,  1.14741911e+02, -5.66415507e+01,
        -8.66837484e+01, -7.70930364e+01, -9.80483564e+01,
         8.16514861e+01,  1.27364697e+01, -1.26196276e+02,
         2.10401251e+01,  2.72363040e+02,  2.03292200e+0

In [27]:
# Calculate the predictions
ypreds = model.predict(X_pre)
ypreds

array([[2972.12057713],
       [3056.63702122],
       [2830.44033178],
       ...,
       [2897.56260776],
       [2712.14659359],
       [2439.69089079]], shape=(191459, 1))

## Evaluate Metrics

In [28]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

mse = mean_squared_error(Y,ypreds)
mae = mean_absolute_error(Y,ypreds)
rmse = mse**(1/2)
r2 = r2_score(Y,ypreds)

# print the evaluation metrics
print(f"MSE : {mse}")
print(f"MAE : {mae}")
print(f"RMSE : {rmse}")
print(f"R2-score : {r2*100:.2f}%")

MSE : 1029644.7717761024
MAE : 695.6719257364293
RMSE : 1014.7141330326007
R2-score : 22.18%
